In [74]:
import pandas as pd
import numpy as np
import cenpy
import os

In [123]:
cen_vars = {
        'B01003_001E': 'population', # total population
        'B06011_001E': 'income', # median household income in USD
        'B17020_002E': 'poverty_total', # number of people (incl. children) living below poverty line
        'B23025_003E': 'labor_force', # number of people who could work
        'B23025_005E': 'unemployed', # number of people in labor force but unemployed
        'B25077_001E': 'house_value', # median value of owner-occupied housing unit
        'B25064_001E': 'rent', # median rent of occupied units
       }

In [124]:
cen = {} 

for year in range(2014, 2022):
    # connect to census api
    api_database = f'ACSDT5Y{year}'
    api_conn = cenpy.remote.APIConnection(api_database)
    g_unit = 'tract'
    g_filter = {'state':'08'} # Colorado
    cen[year] = api_conn.query(cen_vars, geo_unit = g_unit, geo_filter = g_filter)

    # rename columns
    new_cols = list(cen_vars.values())
    new_cols.extend(['state_code', 'county_code', 'tract'])
    cen[year].columns = new_cols

    # calculate unemployment
    cen[year]['unemployment'] = cen[year]['unemployed'].astype(int) / cen[year]['labor_force'].astype(int)

    # calculate poverty rate
    cen[year]['poverty'] = cen[year]['poverty_total'].astype(int) / cen[year]['population'].astype(int)
    
    # drop extraneous cols
    cen[year] = cen[year].drop(['poverty_total', 'labor_force', 'unemployed'], axis = 1)
    
    # write CSV for year
    full_path = os.path.join(os.path.abspath(os.path.dirname('__file__'))) # get file location
    cen[year].to_csv(full_path + f'/census_data/co_{year}.csv')


In [125]:
# just to show a sample:
cen[2014]

,population,income,house_value,rent,state_code,county_code,tract,unemployment,poverty
0,3504,42608,430500,2001,08,035,014506,0.039845,0.057363
1,7359,39786,219200,1738,08,035,014603,0.053281,0.027585
2,4096,52089,276200,1685,08,035,014604,0.060164,0.110107
3,5984,30932,359400,1591,08,037,000301,0.058898,0.052306
4,2289,32500,639000,943,08,037,000302,0.074945,0.069463
...,...,...,...,...,...,...,...,...,...
1244,4287,21353,197000,764,08,029,965200,0.125146,0.193842
1245,3695,20719,114900,735,08,043,978200,0.146419,0.205683
1246,5341,22057,146700,610,08,043,978500,0.048725,0.179367
1247,5165,23180,156600,778,08,043,978300,0.061234,0.204453
